In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Reading the data

df = pd.read_csv('/content/drugLibTrain.csv')

# getting the shapes
print("Shape of train dataset :", df.shape)

Shape of train dataset : (3107, 9)


Exploring the dataset


In [ ]:
#  checking the head of the train dataset
print(df.head())

   Unnamed: 0       urlDrugName  rating         effectiveness  \
0        2202         enalapril       4      Highly Effective   
1        3117  ortho-tri-cyclen       1      Highly Effective   
2        1146           ponstel      10      Highly Effective   
3        3947          prilosec       3  Marginally Effective   
4        1951            lyrica       2  Marginally Effective   

           sideEffects                               condition  \
0    Mild Side Effects  management of congestive heart failure   
1  Severe Side Effects                        birth prevention   
2      No Side Effects                        menstrual cramps   
3    Mild Side Effects                             acid reflux   
4  Severe Side Effects                            fibromyalgia   

                                      benefitsReview  \
0  slowed the progression of left ventricular dys...   
1  Although this type of birth control has more c...   
2  I was used to having cramps so badly that

In [ ]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3107 entries, 0 to 3106
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         3107 non-null   int64 
 1   urlDrugName        3107 non-null   object
 2   rating             3107 non-null   int64 
 3   effectiveness      3107 non-null   object
 4   sideEffects        3107 non-null   object
 5   condition          3106 non-null   object
 6   benefitsReview     3107 non-null   object
 7   sideEffectsReview  3105 non-null   object
 8   commentsReview     3099 non-null   object
dtypes: int64(2), object(7)
memory usage: 218.6+ KB
None


In [ ]:
print(df.describe())

        Unnamed: 0       rating
count  3107.000000  3107.000000
mean   2080.607016     7.006115
std    1187.998828     2.937582
min       0.000000     1.000000
25%    1062.500000     5.000000
50%    2092.000000     8.000000
75%    3092.500000     9.000000
max    4161.000000    10.000000


In [ ]:
# Explore the distribution of ratings
print(df['rating'].value_counts())

10    742
8     558
9     480
7     350
1     305
5     159
6     157
3     146
4     107
2     103
Name: rating, dtype: int64


In [ ]:
# Explore the distribution of side effects
print(df['sideEffects'].value_counts())

Mild Side Effects                1019
No Side Effects                   930
Moderate Side Effects             614
Severe Side Effects               369
Extremely Severe Side Effects     175
Name: sideEffects, dtype: int64


In [ ]:
# Explore the distribution of conditions
print(df['condition'].value_counts())

depression                                          236
acne                                                165
anxiety                                              63
insomnia                                             54
birth control                                        49
                                                   ... 
panic attacks, depression                             1
extrinsic aging                                       1
all over and various type pain                        1
excessive coughing, later diagnosed as pneumonia      1
total hysterctomy                                     1
Name: condition, Length: 1426, dtype: int64


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Fill missing values
df = df.fillna('')

In [ ]:
# Remove duplicates
df = df.drop_duplicates(subset=['urlDrugName', 'condition', 'sideEffects'])

In [ ]:
# Feature engineering
df['effectiveness'] = df['effectiveness'].apply(lambda x: x.replace('No Effect', 'No effect'))

<ipython-input-15-eac502a940b7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['effectiveness'] = df['effectiveness'].apply(lambda x: x.replace('No Effect', 'No effect'))


In [ ]:
# Encode categorical features
cat_cols = ['urlDrugName', 'condition', 'sideEffects', 'effectiveness']
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


<ipython-input-16-e0cfd4fff6ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])
<ipython-input-16-e0cfd4fff6ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])
<ipython-input-16-e0cfd4fff6ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [ ]:
import sklearn
# Text feature extraction
tfidf = TfidfVectorizer(stop_words='english')
text_cols = ['benefitsReview', 'sideEffectsReview', 'commentsReview']
for col in text_cols:
    tfidf_matrix = tfidf.fit_transform(df[col])
    feature_names = tfidf.get_feature_names_out()
    dense_matrix = tfidf_matrix.todense()
    for i, name in enumerate(feature_names):
        col_name = col + '_' + name
        df[col_name] = dense_matrix[:, i]
    df = df.drop(col, axis=1)

Streaming output truncated to the last 5000 lines.
<ipython-input-17-6ed1dec2e941>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = dense_matrix[:, i]
<ipython-input-17-6ed1dec2e941>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = dense_matrix[:, i]
<ipython-input-17-6ed1dec2e941>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead

In [ ]:
# Separate the target variable
X = df.drop(['rating'], axis=1)
y = df['rating']


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Train a random forest regressor on the training data
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict on the testing data
y_pred = rf.predict(X_test)

In [ ]:
# Evaluate the performance of the model
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)


MSE: 2.1636018218623483
RMSE: 1.470918699949915
R-squared: 0.7700895375287606


In [ ]:
def recommend_drugs(condition, benefits, side_effects, effectiveness):
    # Encode the input attributes
    condition = le.transform([condition])[0]
    benefits_matrix = tfidf.transform([benefits])
    side_effects = le.transform([side_effects])[0]
    effectiveness = le.transform([effectiveness])[0]

    # Create a new data point using the encoded attributes and TF-IDF features
    data = {'urlDrugName': [0], 'condition': [condition], 'sideEffects': [side_effects], 'effectiveness': [effectiveness]}
    for i, name in enumerate(feature_names):
        col_name = text_cols[i] + '_' + name
        data[col_name] = [benefits_matrix[0, i]]
    df_new = pd.DataFrame(data)

    # Predict the rating using the trained model
    rating = rf.predict(df_new)

    # Find drugs with similar attributes and high ratings
    df_similar = df[(df['condition'] == condition) & (df['sideEffects'] == side_effects) & (df['effectiveness'] == effectiveness)]
    df_similar = df_similar[df_similar['rating'] >= rating - 1]
    df_similar = df_similar.sort_values(by='rating', ascending=False).head(10)

    # Return the list of recommended drugs
    recommended_drugs = list(df_similar['urlDrugName'].map(le.inverse_transform))
    return recommended_drugs


In [ ]:
# Test the recommendation system
condition = 'depression'
benefits = 'This drug has helped me a lot with my depression symptoms. I feel more energetic and less anxious.'
side_effects = 'Mild'
effectiveness = 'Very effective'
recommended_drugs = recommend_drugs(condition, benefits, side_effects, effectiveness)
print("Recommended drugs:", recommended_drugs)


ValueError: ignored